In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReviewAnalysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [24]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

In [25]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only rows with total vote equal to or greater than 20
filtered_df = vine_df.filter(col("total_votes") >= 20)
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [26]:
# Filter for only rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
filtered_votes_df = filtered_df.filter(col("helpful_votes")/col("total_votes") >= 0.5)
filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [27]:
# Filter for only rows where a review was written as part of the Vine program
filtered_vinesY_df = filtered_votes_df.filter(col("vine") == 'Y')
filtered_vinesY_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R84VUCDBCI29U|          3|           18|         28|   Y|                N|
| R4V3ICFDTIDIF|          4|           20|         21|   Y|                N|
|R1JZ0JAPW83WFS|          4|           54|         58|   Y|                N|
|R1XH7EA97FAVP7|          3|           35|         44|   Y|                N|
|R1482JAU1ZR7QH|          4|           15|         22|   Y|                N|
|R1GGYGVTHP84DG|          4|           25|         27|   Y|                N|
| RXGU9DSKZJSP0|          3|           21|         22|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [28]:
# Filter for only rows where a review was no part of the Vine program
filtered_vinesN_df = filtered_votes_df.filter(col("vine") == 'N')
filtered_vinesN_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [35]:
# Determine the total number of the paid reviews
total_paid_reviews = filtered_vinesY_df.count()
# Determine the number of 5-star paid reviews
paid_reviews_five_stars = filtered_vinesY_df.filter(col("star_rating") == 5).count()
# Determine the percentage of 5-star paid reviews
percentage_five_stars_paid_review = (paid_reviews_five_stars/total_paid_reviews)*0.1
print(f"Total number of paid reviews: {total_paid_reviews}")
print(f"Number of 5-star paid reviews: {paid_reviews_five_stars}")
print(f"Percentage of 5-star paid reviews: {percentage_five_stars_paid_review}")

Total number of paid reviews: 7
Number of 5-star paid reviews: 0
Percentage of 5-star paid reviews: 0.0


In [40]:
# Determine the total number of the unpaid reviews
total_unpaid_reviews = filtered_vinesN_df.count()
# Determine the number of 5-star unpaid reviews
unpaid_reviews_five_stars = filtered_vinesN_df.filter(col("star_rating") == 5).count()
# Determine the percentage of 5-star paid reviews
percentage_five_stars_unpaid_review = (unpaid_reviews_five_stars/total_unpaid_reviews)*0.1
print(f"Total number of unpaid reviews: {total_unpaid_reviews}")
print(f"Number of 5-star unpaid reviews: {unpaid_reviews_five_stars}")
print(f"Percentage of 5-star unpaid reviews: {percentage_five_stars_unpaid_review:.2f}")


Total number of unpaid reviews: 105979
Number of 5-star unpaid reviews: 67580
Percentage of 5-star unpaid reviews: 0.06
